<a href="https://colab.research.google.com/github/handeo/SBTi-finance-tool/blob/main/examples/quick_temp_score_calculation_albaraka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SBTi-Finance Tool - Quick Temperature Score Calculation
This notebook provides a simple example of the SBTi-Finance Tool. It shows how to use it to calculate the temperature score for companies, aggregate them to a portfolio level to get the portfolio temperature score. It also shows you how to calculate the portfolio coverage.

Please see the [methodology](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf), [guidance](https://sciencebasedtargets.org/wp-content/uploads/2020/10/Financial-Sector-Science-Based-Targets-Guidance-Pilot-Version.pdf) and the [technical documentation](https://sciencebasedtargets.github.io/SBTi-finance-tool/) for more details.

See 1_analysis_example (on [Colab](https://colab.research.google.com/github/OFBDABV/SBTi/blob/master/examples/1_analysis_example.ipynb) or [Github](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/master/examples/1_analysis_example.ipynb)) for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular.

### Install the SBTi Python module
This is only required if you have not already installed the module.

In [1]:
%pip install sbti-finance-tool

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 27.7 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.1
    Uninstalling pydantic-2.6.1:
      Successfully uninstalled pydantic-2.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0

In [2]:
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
import pandas as pd

## Download the dummy data provider
We have prepared dummy data for you to be able to run the tool as it is to familiarise yourself with how it works. To use your own data; please check out to the [Data Requirements section](https://sciencebasedtargets.github.io/SBTi/DataRequirements.html) of the technical documentation for more details on data requirements and formatting.

*The dummy data may include some company names, but the data associated with those company names is completely random and any similarities with real world data is purely coincidental.


In [3]:
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/Data_Provider.xlsx"):
    urllib.request.urlretrieve("https://github.com/handeo/SBTi-finance-tool/raw/main/data/Data_Provider.xlsx", "data/Data_Provider.xlsx")
if not os.path.isfile("data/portfolio_data.csv"):
    urllib.request.urlretrieve("https://github.com/handeo/SBTi-finance-tool/raw/main/data/portfolio_data.csv", "data/portfolio_data.csv")

##### Logging
The SBTi module uses the Python standard library logging utilities to send log messages. The log level can be changed according to the user's needs.

In [4]:
import logging
root_logger = logging.getLogger()
root_logger.setLevel("INFO")

## Create a data provider
Data providers let you connect to the data source of your choice. In this case we are connecting to Excel as a data provider. For all available dataproviders check the implementation [here](https://github.com/ScienceBasedTargets/SBTi-finance-tool/tree/master/SBTi/data)

In [6]:
provider = ExcelProvider(path="data/Data_Provider.xlsx")

## Load your portfolio
In our case the portfolio is stored as a CSV file. The portfolio should at least have an "id" (the identifier of the company) and a "proportion" (the weight of the company in your portfolio e.g. the value of the shares you hold) column.

Please see the technical documentation on [Data Legends](https://sciencebasedtargets.github.io/SBTi/Legends.html#) for details on data requirements.

In [7]:
df_portfolio = pd.read_csv("data/portfolio_data.csv", encoding="iso-8859-1")

In [13]:
df_portfolio.head(5)

,company_name,company_id,company_isin,company_lei,weights,investment_value,engagement_target
0,Company 6,Customer6,NaN,NaN,2.86%,3101504,False
1,Company 116,Customer116,NaN,NaN,27.74%,30037648,False
2,Company 117,Customer117,NaN,NaN,19.94%,21590662,False
3,Company 119,Customer119,NaN,789000USXJ5EA7GBM139,13.58%,14708835,False
4,Company 121,Customer121,NaN,789000SVZUQTHE1Q7X60,14.97%,16208861,False


To load the data from the data provider, we have to pass a list of IPortfolioCompany instances. The module has a strict [data model](https://sciencebasedtargets.github.io/SBTi-finance-tool/autoapi/SBTi/interfaces/index.html) to convert Pandas Dataframe to the right object types we supplied a utility function.


In [14]:
companies = SBTi.utils.dataframe_to_portfolio(df_portfolio)

## Calculate the temperature scores
In the amended portfolio you'll find your original portfolio, amended with the emissions and the temperature score.

In [15]:
temperature_score = TemperatureScore(                  # all available options:
    time_frames=list(SBTi.interfaces.ETimeFrames),     # ETimeFrames: SHORT MID and LONG
    scopes=[EScope.S1S2, EScope.S3, EScope.S1S2S3],    # EScopes: S3, S1S2 and S1S2S3
    aggregation_method=PortfolioAggregationMethod.WATS # Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.
)
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies)

Status code from fetching the CTA file: 200, 200 = OK


For every company the tool assigns a score for all the requested timeframe and scope combinations. In this example we used the full set resulting in 9 scores per company as displayed below:

In [16]:
amended_portfolio[['company_name', 'time_frame', 'scope', 'temperature_score']].head(9)

,company_name,time_frame,scope,temperature_score
0,Company 6,SHORT,S1S2,3.2
1,Company 6,SHORT,S3,3.2
2,Company 6,SHORT,S1S2S3,3.2
3,Company 6,MID,S1S2,3.2
4,Company 6,MID,S3,3.2
5,Company 6,MID,S1S2S3,3.2
6,Company 6,LONG,S1S2,3.2
7,Company 6,LONG,S3,3.2
8,Company 6,LONG,S1S2S3,3.2


## Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done using different aggregation methods. Here we'll use the "Weighted Average Temperature Score" (WATS) by initializing the TemperatureScore Object with PortfolioAggregationMethod.WATS. For more details, please refer to notebook 4 (on [Colab](https://colab.research.google.com/github/OFBDABV/SBTi/blob/master/examples/4_portfolio_aggregations.ipynb) or [GitHub](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/master/examples/4_portfolio_aggregations.ipynb)) and the [methodology document](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf) sections 3.2. The temperature scores are calculated per time-frame/scope combination.


In [17]:
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)

In [18]:
# Here we cast a ScoreAggregation object to a Pandas Dataframe for viewing the temp scores in a human readable way
pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))

,short,mid,long
S1S2,3.2,3.2,3.2
S3,3.2,3.2,3.2
S1S2S3,3.2,3.2,3.2


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['short']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'short'}, axis=1)
              .sort_values('short', ascending=True))
  xs = counted['short']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_0.sort_values('short', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('short')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['mid']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'mid'}, axis=1)
              .sort_values('mid', ascending=True))
  xs = counted['mid']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_1.sort_values('mid', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('mid')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['long']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'long'}, axis=1)
              .sort_values('long', ascending=True))
  xs = counted['long']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_2.sort_values('long', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('long')
_ = plt.ylabel('count()')

## Portfolio coverage

The portfolio coverage provides insights in the proportion of the portfolio that has set SBTi-approved GHG emissions reduction targets. Only companies with SBTi-status "Approved" are included in the portfolio coverage.

To calculate the portfolio coverage we use the same aggregation methods we use for the Portfolio Temperature Score. In this example we use the "Weighted Average Temperature Score" (WATS). For more details on aggregation methods and the portfolio coverage method, please refer to the [methodology document](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf) sections 3.2 and also turn to notebook 4 (on [Colab](https://colab.research.google.com/github/OFBDABV/SBTi/blob/master/examples/4_portfolio_aggregations.ipynb) or [GitHub](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/master/examples/4_portfolio_aggregations.ipynb)) for more aggregation examples.

In [19]:
portfolio_coverage_tvp = PortfolioCoverageTVP()
coverage = portfolio_coverage_tvp.get_portfolio_coverage(amended_portfolio.copy(), PortfolioAggregationMethod.WATS)
print("Portfolio coverage is: {c:.2f}%".format(c=coverage))

Portfolio coverage is: 0.00%
